In [1]:
import dai
import json
import pandas as pd
from IPython.display import Markdown
from datetime import datetime, timedelta
from openai import OpenAI

article_prompt = """
* 文章标题: {title}
* 发布时间: {publish_date}
* 发布分类: {category}
* 子分类: {sub_category}
* 内容简介: {brief}
* 正文: {content}
"""

sys_prompt = """
# 角色

你是一位专业的白糖期货研究员助手。你的核心职责是高效处理输入的白糖相关文章，精准总结信息，并基于文章内容给出初步的投资建议方向。

# 核心目标

严格基于输入的文章内容，精准提炼信息，并按照指定格式输出结构化的研究摘要和观点。

# 任务要求

1. 严格分析: 你的分析和建议必须严格基于所提供的文章内容，不引入外部信息或个人过度解读。
2. 精准提炼: 准确识别并提取文章中影响糖价的关键因素（例如：供需状况、主产国天气、政策变动、宏观经济指标、相关市场如原油/外汇动态等）。若文中包含具体的数字（如价格、产量、库存量、百分比变化等），务必在总结中清晰呈现。
3. 全面总结: 针对文章中所有与白糖期货相关、有价值的信息点进行全面而精炼的概括，形成`summary`。
4. 鲜明观点: (初步投资建议): 基于文章内容，大胆推测白糖期货未来可能的走势，给出清晰的思考逻辑链路，或提出有价值的观察点，形成`opinion`，即为初步的投资建议方向。
5. 格式规范: 必须严格按照下方“输出报告”中定义的JSON格式进行输出。所有字段名和结构需完全一致，包括字段顺序。
6. 空内容处理:
    * 如果文章内容不足以生成有意义的`summary`，则`summary`字段的值应为空字符串 `""`。
    * 如果文章内容不足以形成合理的`opinion`（即初步投资建议），则`opinion`字段的值应为空字符串 `""`。
    * `title`字段应始终从原文提取。
    * `date`字段应从原文提取并格式化为 YYYY-mm-dd；若无法提取或原文未提供，则值为 `null`。
7. 字数控制: 输出内容应力求专业、客观、逻辑清晰、简洁明了，字数控制在500字至800字之间。

# 输入文章

输入内容包括：文章标题，发布时间，发布分类，子分类，内容简介，正文。比如：

* 文章标题: 大地期货3.20：白糖早报
* 发布时间: 2025-03-20
* 发布分类: 行业研究 - 日报
* 内容简介: 昨晚ICE糖市主力合约震荡下行；受外盘下跌影响，郑盘主力合约震荡走弱；美元指数震荡上扬
* 正文: 昨晚ICE糖市主力合约震荡下行，最终以19.65美分/磅收盘，下跌0.26美分/磅，近期糖价持续上涨后获利了结巴西较低的降雨量以及印度生产前景恶化带来的利好逐渐消化，警惕糖价技术性回落。美元指数震荡上扬，尾盘回落，美联储维持利率不变并预计增长放缓。美原油及布伦特原油震荡收涨，此前美国政府数据显示燃料油库存减少，但美联储决定维持利率不变，限制了涨幅。郑盘主力合约震荡走弱，最终以6091元/吨收盘，受外盘下跌影响，同时，糖价持续上涨后有回调需求，利好逐渐消化，留意下行风险。

# 输出报告

输出报告以 json 格式输出，请严格遵守

{
    "title": <文章标题，原文的文章标题>,
    "date": <新闻时间，格式为 YYYY-mm-dd，没有请填 null>,
    "summary": <内容总结，根据文章内容总结关键信息>,
    "opinion": <输出观点，根据文章内容阐述对白糖期货未来走势的观点>,
}

比如，输出报告如下：
{
    "title": '大地期货3.20：白糖早报'
    "date": '2025-03-20',
    "summary": 'ICE糖价因获利了结及利好消化下跌；郑糖受外盘拖累走弱。前期上涨后面临技术回调压力，巴西天气及印度产量仍是焦点。',
}

# 例子1

## 输入报告

* 文章标题: 20230331郑商所白糖期货数据
* 发布时间: 2023-03-31 00:00:00
* 发布分类: 国内新闻
* 子分类: 最新资讯
* 内容简介: 20230331郑商所白糖期货数据
* 正文: 交易数据持仓排名仓单日报（责任编辑：Scarlett）

## 输出报告

{
    "title": '20230331郑商所白糖期货数据'
    "date": '2023-03-31',
    "summary": '',
}
"""

client = OpenAI(api_key="sk-7e0d7d183ae84e08b8579a537feff921", base_url="https://api.deepseek.com")

In [2]:
# start_date="2023-04-01"
# end_date="2023-12-31"
today = "2023-04-01"
date_format = "%Y-%m-%d"
start_date = (datetime.strptime(today, date_format) - timedelta(days=7)).strftime(date_format)
end_date = today
data = dai.query("select * from aisugar_hisugar", filters={"date": [start_date, end_date]}).df()
data = data.drop_duplicates(subset=['title', 'brief', 'content'])
data

,article_id,date,category,sub_category,title,brief,content
0,2023032519340845738986,2023-03-26,国际新闻,最新资讯,UNICA：2022/23榨季截至3月上半月，巴西中南部地区累计产糖量为3358.3万吨,3月上半月，巴西中南部地区甘蔗入榨量为60.8万吨，较去年同期的14.2万吨增加了46.6万...,3月上半月，巴西中南部地区甘蔗入榨量为60.8万吨，较去年同期的14.2万吨增加了46.6万...
1,2023032520022103845691,2023-03-26,国际新闻,最新资讯,UNICA：预计巴西中南部地区3月下半月甘蔗压榨量为500万吨,UNICA于3月24日公布的预测显示，预计巴西中南部地区的糖厂将在3月下半月压榨约500万吨...,UNICA于3月24日公布的预测显示，预计巴西中南部地区的糖厂将在3月下半月压榨约500万吨...
2,2023032612370329576461,2023-03-26,国际新闻,最新资讯,20230321 CFTC持仓动态,截至3月21日当周，ICE原糖期货+期权总持仓为1107831手，较前一周减少20722手。,截至3月21日当周，ICE原糖期货+期权总持仓为1107831手，较前一周减少20722手。...
3,2023032708373763174749,2023-03-27,国内新闻,最新资讯,“蔗”里甜如蜜 “链”起转型路 ——西乡塘区坛洛镇推动蔗糖产业稳步发展惠及农户,近日，走进西乡塘区坛洛镇甘蔗种植区，万亩甘蔗林迎风招展，蔗农们忙着砍收甘蔗。一辆辆满载甘蔗的...,近日，走进西乡塘区坛洛镇甘蔗种植区，万亩甘蔗林迎风招展，蔗农们忙着砍收甘蔗。一辆辆满载甘蔗的...
5,2023032708412098534965,2023-03-27,行业研究,日报,中泰期货0327：白糖早报,关注内外价差变化,"上周ICE 原糖反弹,5月合约报收20.84美分/磅，周涨幅 0.87%，不过ICE白糖期..."
...,...,...,...,...,...,...,...
155,2023033109375325894809,2023-03-31,国内新闻,最新资讯,广西融水：春耕春管忙，不负好春光,连日来，在广西柳州市融水苗族自治县各乡镇，村民抢农时、抓生产，田间地头尽是繁忙景象。<br/...,连日来，在广西柳州市融水苗族自治县各乡镇，村民抢农时、抓生产，田间地头尽是繁忙景象。3月28...
157,2023033116170663547507,2023-03-31,国内新闻,最新资讯,20230331国内制糖集团报价,今日广西白糖现货成交价为6346元/吨，上涨107元/吨；产区制糖集团报价区间为6260~6...,今日广西白糖现货成交价为6346元/吨，上涨107元/吨；产区制糖集团报价区间为6260~6...
158,2023033116231644467143,2023-03-31,国内新闻,最新资讯,20230331郑商所白糖期货数据,20230331郑商所白糖期货数据,交易数据持仓排名仓单日报（责任编辑：Scarlett）
160,2023033116370334175955,2023-03-31,国内新闻,最新资讯,关于召开《2023年中国（昆明）食糖市场形势分析会议》的通知,关于召开《2023年中国（昆明）食糖市场形势分析会议》,（责任编辑：Clax）


In [6]:
import os
import numpy as np
import pandas as pd
from datetime import datetime


class DBFile:
    """文件类型数据库"""
    def __init__(self):
        self.parent_path = os.path.dirname(os.path.abspath(__file__))

        self.base_path = self.parent_path + '/DataBase/'
        self.category_structure = {
            "国内新闻": [
                '最新资讯', '糖料生产', '食糖生产', '糖厂开工', '食糖产销',
                '海关数据', '现货报价/进口成本', '食糖消费', '白糖期货/期权',
                '糖业政策', '蔗区气象', '副产品/替代品', '产业风采', '通知/公告', '糖业会议/培训',
            ],
            "国际新闻": [
                '最新资讯', '糖料生产', '食糖生产', '食糖贸易', '食糖消费',
                '供需分析', '期货市场', '糖业政策', '环球财经'
            ],
            "行业研究": [
                '热点研究', '日报', '周报', '月报', '国内市场', '国际市场',
                '调研报告', '宏观研究', '权威解读', '季报年报',
                '糖业论文', '糖业科普',
            ]
        }

    def _validate_data(self, df: pd.DataFrame) -> None:
        """验证数据格式和分类有效性"""
        required_columns = {'article_id', 'date', 'category', 'sub_category', 'title', 'brief', 'content'}
        if not required_columns.issubset(df.columns):
            missing = required_columns - set(df.columns)
            raise ValueError(f"缺少必要列: {missing}")

        # 验证分类有效性
        valid_categories = self.category_structure.keys()
        invalid_categories = df[~df['category'].isin(valid_categories)]
        if not invalid_categories.empty:
            raise ValueError(f"无效分类: {invalid_categories['category'].unique().tolist()}")

        # 验证子分类有效性
        for cat, valid_subs in self.category_structure.items():
            cat_data = df[df['category'] == cat]
            invalid_subs = cat_data[~cat_data['sub_category'].isin(valid_subs)]
            if not invalid_subs.empty:
                raise ValueError(f"分类'{cat}'下无效的子分类: {invalid_subs['sub_category'].unique().tolist()}")

    def save_data(self, df: pd.DataFrame, table: str) -> None:
        """保存DataFrame到指定目录结构"""
        self._validate_data(df)
        
        for (date_val, category, sub_category), group_df in df.groupby(['date', 'category', 'sub_category']):
            dir_path = os.path.join(self.base_path+f"{table}/", str(date_val.strftime("%Y-%m-%d")), category, sub_category)
            os.makedirs(dir_path, exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_path = os.path.join(dir_path, f"data_{timestamp}.parquet")
            group_df.to_parquet(file_path)

DBFile().save_data(data)

NameError: name '__file__' is not defined

In [5]:
result = []
for i in range(0, 3):
    print(f"正在分析第 {i} 篇，总共 {data.shape[0]} 篇")
    article = data.iloc[i].to_dict()
    user_prompt = article_prompt.format(
        title = article["title"],
        publish_date = article["date"],
        category = article["category"],
        sub_category = article["sub_category"],
        brief = article["brief"],
        content = article["content"],
    )
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False,
        response_format={'type': 'json_object'}
    )
    answer = response.choices[0].message.content
    answer = json.loads(answer)
    result.append(answer)

with open("assistant.json", 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

正在分析第 0 篇，总共 100 篇


KeyboardInterrupt: 

In [4]:
data.shape[0]

100